# Song recommender project #

 Garcinuño Feliciano Angela <br>
 Huima Klaara <br>
 Massot Lucas 

In [31]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import axes3d

In [32]:
df = pd.read_csv("output.csv")

In [35]:
df.columns

Index(['artist_name', 'title', 'artist_location', 'release', 'hotttness',
       'familiarity', 'danceability', 'duration', 'energy', 'loudness', 'year',
       'tempo', 'analysis_rate', 'end_of_fade_in', 'key', 'key_confidence',
       'mode', 'mode_confidence', 'start_of_fade_out', 'time_signature',
       'time_signature_conf', 'track_id'],
      dtype='object')